<a href="https://colab.research.google.com/github/sahba-zhsdt/Tools/blob/master/JF_Documentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **JAX-Fluids Installation**

1. run `git clone git@github.com:tumaer/JAXFLUIDS.git` (ssh) or `git clone https://github.com/tumaer/JAXFLUIDS.git` (http)
2. cd into the folder
3. run `git checkout <your branch>`
3. use conda if you don't have root privileges
4. install GPU support JAX
``` bash=
pip install --upgrade pip
# CUDA 12 installation
# Note: wheels only available on linux.
pip install --upgrade "jax[cuda12_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
# CUDA 11 installation
# Note: wheels only available on linux.
pip install --upgrade "jax[cuda11_pip]" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
```
5. create python venv to isolate the project by `python -m venv <name>`
6. activate the python venv by `source <name>/bin/activate`
7. run `pip install --editable .`


> Let's call JAX-Fluids as **JF**

# **Examples**
* Then we'll describe 3 cases from the very beginning to post process stage.\
A. cylinder flow in 2D and lator 3D \
B. bubble collapse in cavitation \
C. Supersonic 2D ramp
* In the end, we'll investigate how to feed them into ML section.

## **Example A.** Cylinderflow Simulation > Karman Vortex Street

See this image [Re and Vortex shedding](https://www.researchgate.net/profile/Aytekin-Duranay/publication/357826451/figure/fig1/AS:1112072591015937@1642150175200/The-relationship-between-Reynolds-number-and-flow-regimes-around-a-fixed-smooth-circular.png): According to the changes in Re number, you can observe vortex shedding in cylinder wake. Vortex shedding also known as Karman Vortex Street is among the most fundamental problems in CFD. Although, simulating cinsiders to be expensive, let's generate initial data to use it in data-driven models.

Since vortex shedding appears in a range of Re numbers, you may define the Re or Re span at the begining

1. you need to create or modify .json files which are the inputs to JF.
2. start with **case_setup.json**

### **A.1. case_setup.json**

Here, some parameters will be defined. According to the examples and `nlfvs/src/jaxfluids/data_types/case_setup/__init__.py` there is a dictionary of existing attributes in class CaseSetup:
1. general_setup
2. restart_setup
3. domain_setup
4. boundary_condition_setup
5. initial_condition_setup
6. material_manager_setup
7. solid_properties_setup
8. particle_properties_setup
9. forcing_setup
10. output_quantities_setup
11. nondimensionalization_parameters*

Each of these have some attributes which will be defined  by user in .json file. we will progress step by step to fill all of the needed attributes:

In [ ]:
# 1. general_setup

class GeneralSetup(NamedTuple):
    case_name: str      # usually name in accordance to your problem at hand; here we name it "karman Re 300"
    end_time: float     # it depends on your project, but let's start with the existing example "900.0"
    end_step: int     # you may choose end_time or end_step as the desired ending point. we have chosen the previous one.
    save_path: str      # usually you can choose "./resluts"
    save_dt: float      # defines how often to save the the data. similar to existing example we choose "1.0"
    save_step: int      # either choose time or step in ending the simulation and saving the data
    save_timestamps: np.ndarray     # A timestamp is a numeric representation of a specific point in time. I have no idea of this at the moment. but you can skip it if you have chosen one of the above options.
    save_start: float     # What is this? :)

In [ ]:
# 2. restart_setup
# when you run the same simulation again after once running it
# I need to ask about it; however, in the existing examples you will see only first two attributes equal to 'false' and a path.

class RestartSetup(NamedTuple):
    flag: bool
    file_path: str      # if you have data, you have the file_path from which you want to continue simulation.
    use_time: bool
    time: float
    is_equal_decomposition: bool

In [ ]:
# 3. domain_setup

class DomainSetup(NamedTuple):
    x: AxisSetup      # >>> a
    y: AxisSetup
    z: AxisSetup
    decomposition: DomainDecompositionSetup     # >>> b
    particle_setup: ParticleSetup     # >>> c
    active_axes: Tuple[str]     # the rest seems to play with axes. we can skip them for now.
    active_axes_indices: Tuple[int]
    inactive_axes: Tuple[str]
    inactive_axes_indices: Tuple[int]
    dim: int      # no idea ?!

# >>> a. AxisSetup

class AxisSetup(NamedTuple):
    cells: int      # you choose the number of cells being considered in solver. it depends on the problem. here, according to the existing example we choose "1200" for x and y. (and for z if it  is 3D)
    range: Tuple      # you choose the spatial range. here we choose [-50.0, 200.0]
    stretching: MeshStretchingSetup     # >>>

# >>> MeshStretchingSetup
# in cases that you need more precision, you may add some stretching mesh to the sensitive region.

class MeshStretchingSetup(NamedTuple):
    type: str     # choose between TUPLE_MESH_STRETCHING_TYPES = ("CHANNEL", "BOUNDARY_LAYER", "PIECEWISE", "BUBBLE_1", "BUBBLE_2") + homogeneous ?? I think it is the default.
    tanh_value: float     # you need it if you choose "CHANNEL" , "BOUNDARY_LAYER"
    ratio_fine_region: float      # you need it if you choose "BUBBLE_1", "BUBBLE_2"
    cells_fine: float     # # you need it if you choose "BUBBLE_1", "BUBBLE_2"
    piecewise_parameters: Tuple[PiecewiseStretchingParameters]      # >>>

# >>> PiecewiseStretchingParameters
# if you choose "PIECEWISE", come here. in our problem we need this type. none from above is suitable.
# in each axis, divide the range to pieces and define how many cells it will stretch and how.
# Strestching means the width of the cell will change. naturally it is dec. const. inc. >>> |   |  | | | | |  |   |    | :)

class PiecewiseStretchingParameters(NamedTuple):
    type: str choose between TUPLE_PIECEWISE_STRETCHING_TYPES = ("DECREASING", "INCREASING", "CONSTANT")
    cells: int
    upper_bound: float
    lower_bound: float

# >>> b. DomainDecompositionSetup
# split the domain (and data) into subdomains that can be solved independently – but require data exchange on the domain boundaries.
# I guess it just solves the axes independant from eachother, which it seems we want it! so we add it with the default int=1.

class DomainDecompositionSetup(NamedTuple):
    split_x: int = 1
    split_y: int = 1
    split_z: int = 1

# >>> c. ParticleSetup
# we don't work with particles. skip this part.

class ParticleSetup(NamedTuple):
    particle_count: int = 0

In [ ]:
# 4. boundary_condition_setup

class BoundaryConditionSetup(NamedTuple):
    primitives: BoundaryConditionsField = None      # define BC for primitve variables: u,v,w,p,rho based on BC field     # >>>
    levelset: BoundaryConditionsField = None      # ok here is where the problem plays an vital role. we have "moving fluid and fixed solid with no deformation" in our problem.
                                                  # it is a special and simplified case of famous "Fluid-Fluid Levelset"
                                                  # so we define this part, too. But what is the meaning?
    particles: BoundaryConditionsField = None     # we skip this. the problem is not about particles.

# >>> BoundaryConditionsField
# pay attention to the writing format in the exampele.

class BoundaryConditionsField(NamedTuple):
    east: Tuple[BoundaryConditionsFace] = None      # >>>
    west: Tuple[BoundaryConditionsFace] = None
    north: Tuple[BoundaryConditionsFace] = None
    south: Tuple[BoundaryConditionsFace] = None
    top: Tuple[BoundaryConditionsFace] = None
    bottom: Tuple[BoundaryConditionsFace] = None

# >>> BoundaryConditionsFace

class BoundaryConditionsFace(NamedTuple):
    boundary_type: str = None     # PRIMITIVES_TYPES = ("ZEROGRADIENT", "SYMMETRY", "PERIODIC", "INACTIVE", "WALL", "ISOTHERMALWALL", "MASSTRANSFERWALL", "ISOTHERMALMASSTRANSFERWALL", "DIRICHLET", "NEUMANN")
                                  # LEVELSET_TYPES = ("ZEROGRADIENT", "SYMMETRY", "PERIODIC", "INACTIVE","DIRICHLET", "REINITIALIZATION")
                                  # PARTICLE_TYPES = ("ZEROGRADIENT", "SYMMETRY", "PERIODIC", "INACTIVE")

    bounding_domain_callable: Callable = None     # what is this? I guess the BC can change and be applicable to a specific part.
    primitives_callable: NamedTuple = None      # if not INACTIVE, ZEROGRADIENT; define primitives at the boundry: u,v,w,p,rho > ofcourse in DIRICHLET they will be constant quantities.
                                                # here, we have DIRICHLET BC in the boundry from which fluid flows. Q: real or non-dimensionalized?
    levelset_callable: Callable = None      # I have no comprehension of this part! what does it mean levelset BC? but we will use it as example.
    wall_velocity_callable: VelocityCallable = None     # these three are needed if "WALL", "ISOTHERMALWALL", "MASSTRANSFERWALL", "ISOTHERMALMASSTRANSFERWALL" are selected.
    wall_temperature_callable: Callable = None          # .
    wall_mass_transfer: WallMassTransferSetup = None    # .
    primitives_table: PrimitivesTable = None      # I think it is just a concatenation of primitives and axes names and values?

#### [ZERO-GRADIENT](https://www.cfdsupport.com/wp-content/uploads/2021/12/75.png)

zeroGradient boundary condition simply extrapolates the quantity to the patch from the nearest cell value. The meaning is, the quantity is developed in space and its gradient is equal to zero in direction perpendicular to the patch (perpendicular to the boundary).

ZEROGRADIENT is a specific NEUMANN condition that imposes zero derivatives. It is just for convenience, i.e., you could impose a ZEROGRADIENT boundary condition with a NEUMANN boundary condition and explicitly set the derivatives to zero for all primitives.

In [ ]:
# 5. initial_condition_setup

class InitialConditionSetup(NamedTuple):
    primitives: NamedTuple      # u,v,w,p,rho
    levelset: InitialConditionLevelset      # >>> a
    solid_velocity: VelocityCallable      # I guess if solid
    particles: InitialConditionParticles      # we skip. no particles
    turbulent: InitialConditionTurbulent      # we skip. no turbulence
    cavitation: InitialConditionCavitation      # we skip. no cavitation
    is_turbulent: bool      # we skip. no turbulence
    is_cavitation: bool     # we skip. no cavitation

# >>> a.
# basically, I have problem in understanding the meaning of IC in levelset.
# BUT, I think we have four options to define from which function the levelset will evolve!
# block - NACA - h5 fiel - levelset user defined function. choose one and continue.
# in the existing example, this inline lambda function is defined: "levelset": "lambda x,y: - 0.5 + jnp.sqrt(x**2 + y**2)"
# I can understand it is a circle (the cross section of the cylinder). so "-0,5" is the distance from free surface?
# what is zero-level of this? how the levels will vary by movment of fluid over this cylinder, then?
# Is not the cross section between moving fluid and the cylinder a curved surface??
# I need to run the file to see the shape!

class InitialConditionLevelset(NamedTuple):
    blocks: Tuple[InitialLevelsetBlock]     # >>>
    levelset_callable: Callable
    h5_file_path: str
    NACA_profile: str     # it is airfoil. used for ramp flow I think
    is_blocks: bool
    is_callable: bool
    is_NACA: bool
    is_h5_file: bool

# >>> InitialLevelsetBlock

class InitialLevelsetBlock(NamedTuple):
    shape: str      # shape_dictionary = {
        # "circle": CircleParameters,
        # "sphere": SphereParameters,
        # "square": SquareParameters,
        # "rectangle": RectangleParameters,
        # "diamond": DiamondParameters,
        # "ellipse": EllipseParameters,
        # "ellipsoid": EllipsoidParameters,
        # }
    parameters: NamedTuple
    bounding_domain_callable: Callable

In [ ]:
# 6. material_properties*

class MaterialManagerSetup(NamedTuple):
    single_material: MaterialPropertiesSetup = None     # >>> a
    levelset_mixture: LevelsetMixtureSetup = None     # >>> b
    diffuse_mixture: DiffuseMixtureSetup = None     # >>> c
    multicomponent_mixture: MultiComponentMixtureSetup = None     # >>> skip for now

# >>> a. MaterialPropertiesSetup

class MaterialPropertiesSetup(NamedTuple):
    """Specifies the properties of a single fluid material.

    :param NamedTuple: _description_
    :type NamedTuple: _type_
    """
    eos: EquationOfStatePropertiesSetup     # >>> a.1
    transport: TransportPropertiesSetup     # >>> a.2

# >>> a.1. EquationOfStatePropertiesSetup
# choose your material. it needs some study to recegnize the right choice depending on the problem.

class EquationOfStatePropertiesSetup(NamedTuple):
    model: str      # DICT_MATERIAL = {
                    # "PerfectGasThermo": PerfectGasThermo,
                    # 'IdealGas': IdealGas,
                    # "IdealGasThermo": IdealGasThermo,
                    # 'SafeIdealGas': SafeIdealGas,
                    # 'StiffenedGas': StiffenedGas,
                    # 'StiffenedGasComplete': StiffenedGasComplete,
                    # 'Tait': Tait,
                    # 'BarotropicCavitationFluid': BarotropicCavitationFluid
                    # }
                    # here let's just choose "IdealGas"
                    # let's see where is SIG and SGC (?)

    ideal_gas_setup: IdealGasSetup      # if IG, continue with this >>> a.1.1 (we will use this in this example)
    ideal_gas_thermo_setup: IdealGasThermoSetup     # if IGT, continue with this >>> a.1.2
    perfect_gas_thermo_setup: PerfectGasThermoSetup     # if PGT, continue with this >>> a.1.3
    stiffened_gas_setup: StiffenedGasSetup      # if SG, continue with this >>> a.1.4
    tait_setup: TaitSetup     # if T, continue with this >>> a.1.5
    barotropic_cavitation_fluid_setup: BarotropicCavitationFluidSetup     # if BCF, continue with this >>> a.1.6

# >>> a.1.1. IdealGasSetup
# in the src, there is a class named IdealGasParameters including only the first two attributes below!
# it seems user can only at least define these two. the rest will be taken care of.

class IdealGasSetup(NamedTuple):
    specific_heat_ratio: float
    specific_gas_constant: float
    cp: float  = None
    T_ref: float = None
    enthalpy_of_formation_molar: float = None
    entropy_of_formation_molar: float = None

# >>> a.1.2. IdealGasThermoSetup

class IdealGasThermoSetup(NamedTuple):
    molar_mass: float
    thermo_model: str
    T_ref: float = None
    nasa7_polynomial: NASAPolynomialSetup = None
    nasa9_polynomial: NASAPolynomialSetup = None

# >>> a.1.3. PerfectGasThermoSetup

class PerfectGasThermoSetup(NamedTuple):
    molar_mass: float
    thermo_model: str
    T_ref: float
    cp: float
    enthalpy_of_formation_molar: float
    entropy_of_formation_molar: float

# >>> a.1.4. StiffenedGasSetup

class StiffenedGasSetup(NamedTuple):
    specific_heat_ratio: float
    specific_gas_constant: float
    background_pressure: float
    energy_translation_factor: float
    thermal_energy_factor: float

# >>> a.1.5. TaitSetup

class TaitSetup(NamedTuple):
    B: float
    N: float
    rho_ref: float
    p_ref: float

# >>> a.1.6. BarotropicCavitationFluidSetup

class BarotropicCavitationFluidSetup(NamedTuple):
    mixture_phase_model: str
    liquid_phase_model: str
    temperature_ref: float
    density_liquid_ref: float
    density_vapor_ref: float
    pressure_ref: float
    speed_of_sound_liquid_ref: float
    speed_of_sound_vapor_ref: float
    speed_of_sound_mixture: float
    enthalpy_of_evaporation_ref: float
    cp_liquid_ref: float
    cp_vapor_ref: float

# >>> a.2. TransportPropertiesSetup

class TransportPropertiesSetup(NamedTuple):
    dynamic_viscosity: DynamicViscositySetup      # >>> a.2.1
    bulk_viscosity: float     # only pass the quantity, depends on the problem but usually in NSE it is zero. in (mu_b = float 0.0)
    thermal_conductivity: ThermalConductivitySetup      # >>> a.2.2
    mass_diffusivity: MassDiffusivitySetup      # >>> a.2.3
    chapman_enskog_parameters: ChapmanEnskogParameters      # why not in the a.2.1 ??

# >>> a.2.1. DynamicViscositySetup
# remember "nu = mu / rho" is "kinematik_viscosity = dynamic_viscosity / density" is " (m^2/s) = (kg / s) / (kg / m^3) "

class DynamicViscositySetup(NamedTuple):
    model: str      # DYNAMIC_VISCOSITY_MODELS = ("CUSTOM", "SUTHERLAND", "CHAPMAN-ENSKOG")
    value: Callable = None      # if "CUSTOM", define a value
    sutherland_parameters: SutherlandParameters = None      # >>> if "SUTHERLAND"

# >>> SutherlandParameters

class SutherlandParameters(NamedTuple):
    """Parameters of the Sutherland model for viscosity
    or thermal conductivity.

    value = value_ref * (T_ref + constant) / (T + constant) * (T / T_ref)**(3/2)

    value_ref: reference viscosity / thermal conductivity
    T_ref: reference temperature [K]
    constant: Sutherland constant [K]

    :param NamedTuple: _description_
    :type NamedTuple: _type_
    """
    value_ref: float
    T_ref: float
    constant: float

# >>> ChapmanEnskogParameters

class ChapmanEnskogParameters(NamedTuple):
    """Parameters for Chapman-Enskog based models for
    transport coefficients.

    collision_diameter: characteristic lenght of the molecule [Angstrom]
    energy_parameter: energy parameter of the Lennard-Jones potential,
        epsilon is the minimum of the pair-potential energy and
        k is Boltzmann's constant (epsilon/k) [K]

    :param NamedTuple: _description_
    :type NamedTuple: _type_
    """
    collision_diameter: float
    energy_parameter: float

# >>> a.2.2. ThermalConductivitySetup

class ThermalConductivitySetup(NamedTuple):
    model: str      # THERMAL_CONDUCTIVITY_MODELS = ("CUSTOM", "SUTHERLAND", "CHAPMAN-ENSKOG", "PRANDTL")
    value: Callable = None
    prandtl_number: float = None
    sutherland_parameters: SutherlandParameters = None

# >>> a.2.3. MassDiffusivitySetup

class MassDiffusivitySetup(NamedTuple):
    model: str      # MASS_DIFFUSIVITY_MODELS = ("CUSTOM", "CHAPMAN-ENSKOG")
    value: Dict = None

# >>> b. LevelsetMixtureSetup
# I guess it essentially is for Fluid-Fluid levelset
# for the problem at hand (karman) we can skip it

class LevelsetMixtureSetup(NamedTuple):
    """Describes the properties of a level-set material mixture,
    containing of a positive and a negative fluid.

    :param NamedTuple: _description_
    :type NamedTuple: _type_
    """
    positive_fluid: MaterialPropertiesSetup     # each is considered as single fluid
    negative_fluid: MaterialPropertiesSetup     # .
    pairing_properties: MaterialPairingProperties     # >>>

# >>> MaterialPairingProperties

class MaterialPairingProperties(NamedTuple):
    surface_tension_coefficient: float

# >>> c. DiffuseMixtureSetup

class DiffuseMixtureSetup(NamedTuple):
    """Describes the material properties of a diffuse mixture
    consisting of N phases (materials).

    fluids: NamedTuple
    pairing: int

    :param NamedTuple: _description_
    :type NamedTuple: _type_
    """
    fluids: NamedTuple
    pairing_properties: MaterialPairingProperties

In [ ]:
# 7. solid_properties
# it is for the Fluid-Solid-Dynamic I guess.
# we skip it for the problem at hand. in karman, solid is fixed.

class SolidPropertiesSetup(NamedTuple):
    velocity: SolidVelocitySetup
    temperature: SolidTemperatureSetup
    density: float

In [ ]:
# 8. particle_properties
# we skip for the problem at hand. we do not deal with particles in karman

class ParticlePropertiesSetup(NamedTuple):
    bubble_content: str
    polytropic_index: float
    vapor_properties: VaporProperties
    non_condensible_gas_properties: NonCondensibleGasProperties
    pairing_properties: ParticleFluidPairingProperties

In [ ]:
# 9. forcing
# I can understand some points but not all.
# need study!

class ForcingSetup(NamedTuple):
    gravity: jnp.ndarray
    mass_flow_target: Callable
    mass_flow_direction: str
    temperature_target: Callable      # in the existing example, only this is used in .json file and is equal to 1.0. what does it mean??? target? kind of constraint?
    hit_forcing_cutoff: int
    geometric_source: GeometricSourceSetup
    acoustic_forcing: AcousticForcingSetup
    custom_forcing: NamedTuple

In [ ]:
# 10. output
# it seems you choose to see which variables?
# can not finr DICT ???

class OutputQuantitiesSetup(NamedTuple):
    primitives: Tuple[str]
    conservatives: Tuple[str]
    real_fluid: Tuple[str]
    levelset: Tuple[str]
    particles: Tuple[str]
    miscellaneous: Tuple[str]
    forcings: Tuple[str]

In [ ]:
# 11. nondimensionalization_parameters
# I do not know why it is not in the main DICT! I saw in example.
# it is used as default with 1.0. I think when using absolute values, these are 1.0

class NondimensionalizationParameters(NamedTuple):
    density_reference: float = 1.0
    length_reference: float = 1.0
    velocity_reference: float = 1.0
    temperature_reference: float = 1.0

In [ ]:
# so your case_satup.json looks like:

"general": {
        "case_name": "karman Re 300",
        "end_time": 900.0,
        "save_path": "./results",
        "save_dt": 1.0
    },
"restart": {
        "flag": false,
        "file_path": "results/cylinderflow/domain/XXXXX.h5" ??
    },
"domain": {
        "x": {
            "cells": 1200,
            "range": [
                -50.0,
                200.0
            ],
            "stretching": {
                "type": "PIECEWISE",
                "parameters": [
                    {
                    "type": "DECREASING",
                    "lower_bound": -50.0,
                    "upper_bound": -4.0,
                    "cells": 200
                    },
                    {
                    "type": "CONSTANT",
                    "lower_bound": -4.0,
                    "upper_bound": 4.0,
                    "cells": 600
                    },
                    {
                    "type": "INCREASING",
                    "lower_bound": 4.0,
                    "upper_bound": 200.0,
                    "cells": 400
                    }
                ]
            }
        },
        "y": {
            "cells": 1200,
            "range": [
                -100.0,
                100.0
            ],
            "stretching": {
                "type": "PIECEWISE",
                "parameters": [
                    {
                    "type": "DECREASING",
                    "lower_bound": -100.0,
                    "upper_bound": -4.0,
                    "cells": 300
                    },
                    {
                    "type": "CONSTANT",
                    "lower_bound": -4.0,
                    "upper_bound": 4.0,
                    "cells": 600
                    },
                    {
                    "type": "INCREASING",
                    "lower_bound": 4.0,
                    "upper_bound": 100.0,
                    "cells": 300
                    }
                ]
            }
        },
        "z": {
            "cells": 1,
            "range": [
                0.0,
                1.0
            ]
        },
        "decomposition": {
            "split_x": 1,
            "split_y": 1,
            "split_z": 1
        }
    },
"boundary_conditions": {
        "primitives": {
            "east": {"type": "ZEROGRADIENT"},
            "west": {
                "type": "DIRICHLET",
                "primitives_callable": {
                    "rho": 1.0,
                    "u": 0.23664319132398464,     # 1. why this?  2. what about non.dimensionalization?
                    "v": 0.0,
                    "w": 0.0,
                    "p": 1.0
                }
            },
            "north": {"type": "ZEROGRADIENT"},
            "south": {"type": "ZEROGRADIENT"},
            "top": {"type": "INACTIVE"},
            "bottom": {"type": "INACTIVE"}
        },
        "levelset": {
            "east": {"type": "ZEROGRADIENT"},
            "west": {"type": "ZEROGRADIENT"},
            "north": {"type": "ZEROGRADIENT"},
            "south": {"type": "ZEROGRADIENT"},
            "top": {"type": "INACTIVE"},
            "bottom": {"type": "INACTIVE"}
        }
    },
"initial_condition": {
        "primitives": {
            "rho": 1.0,
            "u": 0.23664319132398464,
            "v": 0.0,
            "w": 0.0,
            "p": 1.0
        },
        "levelset": "lambda x,y: - 0.5 + jnp.sqrt(x**2 + y**2)"
    },
"material_properties": {
        "equation_of_state": {
            "model": "IdealGas",
            "specific_heat_ratio": 1.4,
            "specific_gas_constant": 1.0
        },
        "transport": {
            "dynamic_viscosity": {
                "model": "CUSTOM",
                "value": 0.0011832159566199233      # why? :)))
            },
            "bulk_viscosity": 0.0,
            "thermal_conductivity": {
                "model": "CUSTOM",
                "value": 0.0
            }
        }
    },
"output": {
        "primitives": ["density", "velocity", "pressure"],
        "levelset": ["levelset", "volume_fraction"]
    },
"nondimensionalization_parameters": {
        "density_reference": 1.0,
        "length_reference": 1.0,
        "velocity_reference": 1.0,
        "temperature_reference": 1.0
    }

### **A.2. numerical_setup.json**

This file will define how the problem will be solved. according to `nlfvs/src/jaxfluids/data_types/numerical_setup/__init__.py` this file will be structured below:
1. conservatives
2. levelset
3. diffuse_interface
4. multicomponent
5. particles
6. cavitation
7. combustion
8. active_physics
9. active_forcings
10. turbulence_statistics
11. output
12. precision

here we define how the problem is solved in solver based on FVM. Let's fill this .json file step by step.

In [ ]:
# 1. conservatives

class ConservativesSetup(NamedTuple):
    halo_cells: jnp.int32
    time_integration: TimeIntegrationSetup      # >>> a
    convective_fluxes: ConvectiveFluxesSetup      # >>> b
    dissipative_fluxes: DissipativeFluxesSetup
    positivity: PositivitySetup

# >>> a. TimeIntegrationSetup

class TimeIntegrationSetup(NamedTuple):
    integrator: TimeIntegrator      # DICT_TIME_INTEGRATION = {'EULER': Euler, 'RK2': RungeKutta2, 'RK3': RungeKutta3, 'RK2_LS4': RungeKutta2_LS4 }
                                    # from nlfvs/src/jaxfluids/time_integration
                                    # RKs are TVD by default. for this problem let's choose it RK3.
    CFL: float      # CFL = velocity * delta_t \ delta_x
    fixed_timestep: float     # on existing example, it is skipped. we let solver choose delta_t?

# >>> b. ConvectiveFluxesSetup
# the left-hand side of the Eq.

class ConvectiveFluxesSetup(NamedTuple):
    convective_solver: ConvectiveFluxSolver     # DICT_CONVECTIVE_SOLVER = {
                                                # 'GODUNOV': HighOrderGodunov,
                                                # 'FLUX-SPLITTING': FluxSplittingScheme,
                                                # 'ALDM': ALDM, >>> for turbulence, we skip.
                                                #-------------------------#
                                                # "ALDMNN"        : ALDMNN,
                                                # "NNSCHEME_V0"   : NNSCHEME_V0,
                                                # }
    riemann_solver: RiemannSolver
    signal_speed: Callable
    reconstruction_stencil: Union[str, SpatialReconstruction]
    split_reconstruction: SplitReconstructionSetup
    flux_splitting: str
    reconstruction_variable: str
    frozen_state: str
    iles_setup: ILESSetup = None
    catum_setup: CATUMSetup = None



### 1.3. convective_fluxes: ConvectiveFluxesSetup
left-hand side of the Eq.

`class ConvectiveFluxesSetup(NamedTuple)`:\
    **convective_solver**: ConvectiveFluxSolver\
    There are three general options for the convective flux function.
1. High-order Godunov Scheme (discontinuity, shock waves, ..)
* The HighOrderGodunov class implements the flux calculation
    according to the high-order Godunov approach in the finite volume
    framework.\
    The calculation of the fluxes consists of three steps:\
    a. RECONSTRUCT STATE ON CELL FACE\
    b. CONVERT PRIMITIVES TO CONSERVATIVES AND VICE VERSA (whyyyyyy?)\
    c. SOLVE RIEMANN PROBLEM (approximately)

> The reconstruction step can be done on primitive or conservative variables in either physical or characteristic space. (**4 choice for recunstruction variable then**)


2. Flux-splitting Scheme
* Base class for the Flux-Splitting Scheme. The flux-splitting schemes transforms conservative variables and physical fluxes into the characteristic space, performs a flux-splitting in characteristic space, and transforms the final flux back to physical space.

> The eigenvalues - which are used according to the user-specified flux-splitting - determine the numerical flux. (**flux_splitting** argument in the later lines is for this.) *Read Toro*


3. ALDM Scheme (for turbulence)

(for 1) riemann_solver: RiemannSolver\
(For Riemann solver) catum_setup: CATUMSetup = None : *it needs 2 more data* \
(for Reimann solver) signal_speed: Callable\
(for 1 and ?) reconstruction_variable: str # PRIMITIVES and other 3\
(for 1 and ?) reconstruction_stencil: Union[str, SpatialReconstruction] #(for flux and mesh?) check for adaptive mesh! and req. halos\
(for 1 and ?) split_reconstruction: SplitReconstructionSetup (?)\
(for 2) flux_splitting: str \
(for ?) frozen_state: str\
(for 3) iles_setup: ILESSetup = None


### **Questions**
* save_start: float     # What is this? :)
* save_timestamps: np.ndarray     # A timestamp is a numeric representation of a specific point in time. I have no idea of this at the moment. but you can skip it if you have chosen one of the above options.
* restart mechanism!
* homogeneous ??
* Strestching means the width of the cell will change. naturally it is dec. const. inc. >>> |   |  | | | | |  |   |    | :)
* ``` bash =
active_axes: Tuple[str]     # the rest seems to play with axes. we can skip them for now.
active_axes_indices: Tuple[int]
inactive_axes: Tuple[str]
inactive_axes_indices: Tuple[int]
dim: int      # no idea ?!
```
* levelset: BoundaryConditionsField meaning
* Q: real or non-dimensionalized? in primitive collable

### **To Study**

* L2 norm error

### riemann_solver: RiemannSolver
the approximate riemann solvers, each with a specific numerical dissipation\

DICT_RIEMANN_SOLVER =\
    'LAX-FRIEDRICHS'\
    'HLL'\
    'HLLC'\
    'HLLC_SIMPLEALPHA'\
    'HLLC-LM'\
    'RUSANOV'\
    'AUSMP'\
    'CATUM' for cavitation from (TUM)

### signal_speed: Callable

DICT_SIGNAL_SPEEDS =\
    'ARITHMETIC'\
    'RUSANOV'\
    'DAVIS'\
    'DAVIS2'\
    'EINFELDT'\
    'TORO'

### catum_setup: CATUMSetup = None :

*it needs 2 more data: example* \
*"transport_velocity": "EGERER",*\
*"minimum_speed_of_sound": 1e-3*\

TUPLE_CATUM_TRANSPORT_VELOCITIES = (\
    "EGERER", "SCHMIDT", "SEZAL", "MIHATSCH", "KYRIAZIS")\

There are five different transport velocity formulations to choose from:\
**EGERER**
        Egerer et al. (2016) - Efficient implicit LES method
        for the simulation of *turbulent cavitating flows*
    
**SCHMIDT**
        Schmidt (2006) - A low Mach number consistent
        compressible approach for simulation of cavitating flows
    
**SEZAL**
        Sezal (2009) - Compressible Dynamics of Cavitating
        3-D Multi-Phase Flows
    
**MIHATSCH**
        Mihatsch (2017) - Numerical Prediction of Erosion
        and Degassing Effects in CavitatingFlows
    
**KYRIAZIS**
        Kyriazis (2017) - Numerical investigation of
        bubbledynamics using tabulated data




### reconstruction_stencil (of atate)

reconstruction_stencil: Union[str, SpatialReconstruction]

a simple argument or split\
if split ("SPLIT-RECONSTRUCTION")\
`class SplitReconstructionSetup(NamedTuple)`:\
**density**: SpatialReconstruction\
**velocity**: SpatialReconstruction\
**pressure**: SpatialReconstruction\
**volume_fraction**: SpatialReconstruction = None\

the dict:\
DICT_SPATIAL_RECONSTRUCTION: Dict[str, SpatialReconstruction] = {\
    "WENO1":            WENO1,\
    "WENO3-JS":         WENO3,\
    "WENO3-JS-ADAP":    WENO3ADAP,\
    "WENO3-N":          WENO3N,\
    "WENO3-NN-OPT1":    WENO3NNOPT1,\
    "WENO3-NN-OPT2":    WENO3NNOPT2,\
    "WENO3-Z":          WENO3Z,\
    "WENO3-Z-ADAP":     WENO3ZADAP,\
    "WENO3-FP":         WENO3FP,\
    "WENO5-JS":         WENO5,\
    "WENO5-JS-ADAP":    WENO5ADAP,\
    "WENO5-Z":          WENO5Z,\
    "WENO5-Z-ADAP":     WENO5ZADAP,\
    "WENO6-CU":         WENO6CU,\
    "WENO6-CU-ADAP":    WENO6CUADAP,\
    "WENO6-CUM1":       WENO6CUM1,\
    "WENO6-CUM2":       WENO6CUM2,\
    "WENO6-CUMD":       WENO6CUMD,\
    "WENO7-JS":         WENO7,\
    "WENO9-JS":         WENO9,\
    "TENO5":            TENO5,\
    "TENO5-A":          TENO5A,\
    "TENO5-ADAP":       TENO5ADAP,\
    "TENO6":            TENO6,\
    "TENO6-ADAP":       TENO6ADAP,\
    "TENO6-A":          TENO6A,\
    "TENO6-A-ADAP":     TENO6AADAP,\
    "TENO8":            TENO8,\
    "TENO8-A":          TENO8A,\
>"MUSCL3":           MUSCL3,\

  "KOREN":            KOREN,\
    "MC":               MC,\
    "MINMOD":           MINMOD,\
    "SUPERBEE":         SUPERBEE,\
    "VANALBADA":        VANALBADA,\
    "VANLEER":          VANLEER,\
    "KOREN-ADAP":       KORENADAP,\
    "MC-ADAP":          MCADAP,\
    "MINMOD-ADAP":      MINMODADAP,\
    "SUPERBEE-ADAP":    SUPERBEEADAP,\
    "VANALBADA-ADAP":   VANALBADAADAP,\
    "VANLEER-ADAP":     VANLEERADAP,

> iHint: MUSCL and (W/T)ENO are for reconstruction.\
check for the "ADAP" for adaptive/stretching mesh. reconstruction depends on mesh!

4. dissipative_fluxes: DissipativeFluxesSetup

left-hand side of the Eq. including diffusion terms by viscosity which have derivatives.
\
*Read this part of Toro*

`class DissipativeFluxesSetup(NamedTuple)`:
1. reconstruction_stencil: SpatialReconstruction
2. derivative_stencil_center: SpatialDerivative
3. derivative_stencil_face: SpatialDerivative\
DICT_DERIVATIVE_FACE: Dict[str, SpatialDerivative] = \
  "CENTRAL2":        \
  "CENTRAL2_ADAP":   \
    "CENTRAL4":     \
    "CENTRAL4_ADAP":    \
    "CENTRAL6":        \
    "CENTRAL6_ADAP":   \
    "CENTRAL8":   \
    "CENTRAL8_ADAP":  \


> iHint: ofcourse, ADAP are for stretching mesh, but in example they did not use it. ??? maybe it is about the second order!! look at below. check it~~


DICT_FIRST_DERIVATIVE_CENTER: Dict[str, SpatialDerivative] = {\
    "CENTRAL2":        \
    "CENTRAL2_ADAP":  \
    "CENTRAL4":        \
    "CENTRAL4_ADAP":   \
    "CENTRAL6":       \
    "CENTRAL6_ADAP":   \
    "CENTRAL8":      \
    "CENTRAL8_ADAP":  \
}

DICT_SECOND_DERIVATIVE_CENTER: Dict[str, SpatialDerivative] = {\
    "CENTRAL2": \
    "CENTRAL4": \
}

5. positivity: PositivitySetup

it is to make sure no nonphysical negatives will not appear. by implementing some limiters\
`class PositivitySetup(NamedTuple)`:\
    flux_limiter: Union[str, bool]\
    is_interpolation_limiter: bool\
    is_thinc_interpolation_limiter: bool\
    is_volume_fraction_limiter: bool\
    is_acdi_flux_limiter: bool\
    is_logging: bool

In [ ]:
1. conservatives: ConservativesSetup

class ConservativesSetup(NamedTuple):
    halo_cells: jnp.int32
    time_integration: TimeIntegrationSetup
    convective_fluxes: ConvectiveFluxesSetup
    dissipative_fluxes: DissipativeFluxesSetup
    positivity: PositivitySetup

SyntaxError: invalid syntax (<ipython-input-1-db9efc37a5e3>, line 1)

In [ ]:
> convective_fluxes: ConvectiveFluxesSetup

class ConvectiveFluxesSetup(NamedTuple):
    convective_solver: ConvectiveFluxSolver
    riemann_solver: RiemannSolver
    signal_speed: Callable
    reconstruction_stencil: Union[str, SpatialReconstruction] #(for flux and mesh?) check for adaptive mesh! and req. halos
    split_reconstruction: SplitReconstructionSetup (?)
    # flux_splitting: str (?)
    reconstruction_variable: str # PRIMITIVES or ?
    # frozen_state: str
    # iles_setup: ILESSetup = None
    catum_setup: CATUMSetup = None

In [ ]:
> dissipative_fluxes: DissipativeFluxesSetup

class DissipativeFluxesSetup(NamedTuple):
    reconstruction_stencil: SpatialReconstruction
    derivative_stencil_center: SpatialDerivative
    derivative_stencil_face: SpatialDerivative